In [ ]:
profile = raw_input("Profile name (e.g. ethan):")

result_filename = raw_input("Company Name (e.g. linkedin):")

In [ ]:
if profile == 'ethan' :
    template_location = ''
    result_location = 'C:\\Users\\eandrian\\Downloads\\'
    jar_location = 'C:\\Users\\eandrian\\Documents\\Jarfolder\\daptor-jdbc-0.9.1-standalone.jar'
    print ('Loading Ethan\'s profile')

elif profile == 'karmen' :
    template_location = 'C:\\Users\\kachong\\OneDrive - Microsoft\\Notebooks\\Pilot_Review.pptx'
    result_location = 'C:\\Users\\kachong\\Downloads\\'
    jar_location = 'C:\\Users\\kachong\\Documents\\jar_files\\daptor-jdbc-0.9.1-standalone.jar'
    print ('Loading Karmen\'s profile')

elif profile == 'sabina' :
    template_location = ''
    result_location = 'C:\\Users\\ssobinin\\Downloads\\'
    jar_location = 'C:\\Users\\ssobinin\\daptor-jdbc-0.9.1-standalone.jar'
    print ('Loading Sabina\'s profile')
        
else :
    print ('Profile not found')

In [ ]:
import pyodbc
import getpass
import pandas
import matplotlib as mpl
import numpy as np

# import libraries

username = raw_input("Teradata Username:")

password = getpass.getpass(prompt='Teradata password:')


In [ ]:
pyodbc.pooling = False 

 
cxn = pyodbc.connect("DRIVER={Teradata};\
DBCNAME=dwprod1.corp.linkedin.com;\
DATABASE=DWH;\
AUTHENTICATION=LDAP;\
UID=%s;\
PWD=%s"\
%(username,password),\
autocommit = True,\
 ANSI = True)

In [ ]:
%matplotlib inline
from linkedin.presto_daptor.presto_setup import get_presto_connection
import matplotlib
import matplotlib.pyplot as plt
import pandas
matplotlib.style.use('ggplot')

daptor_url = 'jdbc:daptor://ltx1-spidergw01.grid.linkedin.com:44444/hive;ssl=true;'
presto_conn = get_presto_connection(daptor_url, jar_location)

In [ ]:
contract = raw_input("contract ids (e.g. 'CS2199415-16'):") 

dashboard = raw_input("dashboard ids (comma separated), optional:") 

country = raw_input("country_sks (comma separated), optional:") 
CS3801856-17
continent = raw_input("continent_sks (comma separated, optional):") 

company_id = raw_input("company_id (comma separated, required):")

In [ ]:
#check if inputs are empty. if they are then comment them out of the target sql query

if country == "" :
    country_filter = "--"
else :
    country_filter = ""
    
if continent == "" :
    continent_filter = "--"
else : 
    continent_filter = ""
    
if contract == "" :
    dashboard_id_filter = ""
    contract_id_filter = "--"
    
elif contract == "" and dashboard == "" :
    print "No Contract or Dashboard ID provided"

else :
    dashboard_id_filter = "--"
    contract_id_filter = ""

# Only run after generating first report of this session

sql_drop1 = """
drop table all_sp;
"""

sql_drop2 = """
drop table all_sn;
"""

sql_drop3 = """
drop table combined_sp;
"""

In [ ]:
sql0_seats = """create volatile table all_sn as (
select lss_contract_id
, substr(SF_CONTRACT_NUMBER,1,12) AS sf_contract
, a.lss_seat_id
, a.member_id as member_sk 
, a.ONBOARD_COMPLETE_DT 
, current_date - a.onboard_complete_dt as days_onboarded
, a.company_sk
, b.country_sk
, case  when sum(input_companysize_cnt + input_functions_cnt + input_industries_cnt + input_regions_cnt + input_seniorities_cnt) >= 1 
        then 1
        else 0
        end as sales_pref_set

from dwh.v_dim_lss_seat a

inner join dwh.v_dim_member_flat b
on a.member_id = b.member_sk 

inner join dwh.v_dim_country c
on b.country_sk = c.country_sk

where a.ONBOARD_COMPLETE_DT is not null
{0} and a.sf_contract_number in ({1})
{2} and a.lss_contract_id IN ({3}) -- parameter
{4} and b.country_sk in ({5})
{6} and c.continent_sk in ({7})
    and current_flag = 'Y'
group by 1, 2, 3, 4, 5, 6, 7, 8
)   
with data primary index (lss_seat_id) 
on commit preserve rows;""".format(contract_id_filter, contract, dashboard_id_filter, dashboard, country_filter, 
                                   country, continent_filter, continent)

print sql0_seats

pandas.io.sql.execute(sql0_seats, cxn)

#Seat Count

sql0_count = """select count(distinct member_sk) as num_sn from all_sn"""

get_seat_count = pandas.read_sql(sql0_count, cxn)

get_seat_count = get_seat_count.replace(np.nan, 0)

get_seat_count.head()


In [ ]:
sql1_all_sp = """
create volatile table all_sp as (
    select distinct a.member_sk
    from dwh.v_dim_sales_member a
        inner join dwh.v_dim_member b
        on a.member_sk = b.member_sk
        
        inner join dwh.v_dim_country c
        on b.country_sk = c.country_sk
        
        inner join dwh.v_dim_company d
        on a.std_company_sk = d.company_sk
        
    where d.company_id in ({0})
        {1} and c.country_sk in ({2})
        {3} and c.continent_sk in ({4})
)
with data primary index (member_sk)
on commit preserve rows
;
""".format(company_id, country_filter, country, continent_filter, continent)

pandas.io.sql.execute(sql1_all_sp, cxn)

sql1_count = """select count(distinct member_sk) from all_sp"""

get_all_sp = pandas.read_sql(sql1_count, cxn)

get_all_sp.head()

In [ ]:
sql2_combined_sp = """
create volatile table combined_sp as (
    select distinct member_sk
    from (
        select distinct member_sk
        from all_sn
        union
        select distinct member_sk
        from all_sp
    ) a
)
with data primary index (member_sk)
on commit preserve rows
;
"""

pandas.io.sql.execute(sql2_combined_sp, cxn)

sql2_count = """select count(distinct member_sk) as num_sp from combined_sp"""

get_combined_sp = pandas.read_sql(sql2_count, cxn)

num_sp = get_combined_sp["num_sp"].iloc[0]

print num_sp


In [ ]:
sql3_all_emp = """
create multiset volatile table all_emp as (
    select p.member_sk
    from dwh.v_dim_position p

    inner join dm_stt.v_om_segment o 
        on p.member_sk = o.member_sk
    
    inner join dwh.v_dim_company c
    on p.std_company_sk = c.company_sk

    where p.is_final_company = 'y'
        and p.is_final_position = 'y'
        and p.end_date is null
        and c.company_id in ({0})
)
with data primary index (member_sk) 
on commit preserve rows;

""".format(company_id)

print sql3_all_emp

pandas.io.sql.execute(sql3_all_emp, cxn)

sql3_count = """select count(distinct member_sk) from all_sp"""

get_all_emp = pandas.read_sql(sql3_count, cxn)

get_all_emp.head()

In [ ]:
sql4_tl_emp = """
create volatile table tl_emp as (
    select member_sk
    from (
        select distinct e.member_sk
            , count(dest_member_sk) as dm_count
            , row_number () over (order by dm_count desc) as "dm_conn"
        from all_emp e
        inner join dwh.v_fact_detail_connections c 
        on c.source_member_sk = e.member_sk

        inner join dm_stt.v_om_segment o 
        on o.member_sk = c.dest_member_sk

        where o.seniority_2_sk >= 6
            and e.member_sk not in (select distinct member_sk from all_sp)
            and c.active ='y'
        group by 1
    ) a
    where dm_conn <= 1000
)
with data primary index (member_sk)
on commit preserve rows;
"""

pandas.io.sql.execute(sql4_tl_emp, cxn)

sql4_count = """select count(distinct member_sk) from tl_emp"""

get_tl_emp = pandas.read_sql(sql4_count, cxn)

get_tl_emp.head()

In [ ]:
sql5_sn_cxns = """
create multiset volatile table sn_cxns as (
    select c.source_member_sk as member_sk
        , c.dest_member_sk
        , c.date_sk
        , b.seniority_2_sk
    from dwh.v_fact_detail_connections c
    inner join all_sn a 
    on c.source_member_sk = a.member_sk
            
    left join dm_stt.v_om_segment b
    on c.dest_member_sk = b.member_sk
    
    where c.active = 'y'
    
) 
with data primary index (member_sk, dest_member_sk) 
on commit preserve rows
;
"""

pandas.io.sql.execute(sql5_sn_cxns, cxn)

sql5_count = """
select count(distinct dest_member_sk) as sn_cxns 
    , count(distinct case when a.seniority_2_sk >= 6 then dest_member_sk end) as sn_dm_cxns
from sn_cxns a

"""

get_sn_cxns = pandas.read_sql(sql5_count, cxn)

get_sn_cxns.head()

In [ ]:
sql6_com_sp_cxns = """
create multiset volatile table com_sp_cxns as (
    select c.source_member_sk as member_sk
        , c.dest_member_sk
        , c.date_sk
        , b.seniority_2_sk
    from dwh.v_fact_detail_connections c
    inner join combined_sp a 
    on c.source_member_sk = a.member_sk
    
    left join dm_stt.v_om_segment b
    on c.dest_member_sk = b.member_sk

    where c.active = 'y'
) 
with data primary index (member_sk, dest_member_sk) 
on commit preserve rows
;
"""

pandas.io.sql.execute(sql6_com_sp_cxns, cxn)

sql6_count = """
select count(distinct dest_member_sk) as com_sp_cxns 
    , count(distinct case when a.seniority_2_sk >= 6 then dest_member_sk end) as com_sp_dm_cxns
from com_sp_cxns a


"""

sql6_count_dms = """
select count(distinct dest_member_sk)
from com_sp_cxns
where seniority_2_sk >= 6"""

get_com_sp_cxns = pandas.read_sql(sql6_count, cxn)

get_com_sp_cxns.head()

In [ ]:
sql7_all_emp_cxns = """
create multiset volatile table all_emp_cxns as (
    select c.source_member_sk as member_sk
        , c.dest_member_sk
        , c.date_sk
        , b.seniority_2_sk
    from dwh.v_fact_detail_connections c
    inner join all_emp a 
    on c.source_member_sk = a.member_sk
    
    left join dm_stt.v_om_segment b
    on c.dest_member_sk = b.member_sk
    where c.active = 'y'
) 
with data primary index (member_sk, dest_member_sk) 
on commit preserve rows

"""

pandas.io.sql.execute(sql7_all_emp_cxns, cxn)

sql7_count = """
select count(distinct dest_member_sk) as all_emp_cxns 
    , count(distinct case when a.seniority_2_sk >= 6 then dest_member_sk end) as all_emp_dm_cxns
from all_emp_cxns a

"""

get_all_emp_cxns = pandas.read_sql(sql7_count, cxn)

get_all_emp_cxns.head()

In [ ]:
sql8_tl_extend = """
create multiset volatile table teamlink_extend as (
select a.member_sk
    , a.dest_member_sk
    , a.date_sk
    , a.seniority_2_sk
from com_sp_cxns a

union 

select a.member_sk
    , a.dest_member_sk
    , a.date_sk
    , a.seniority_2_sk
from all_emp_cxns a

)
with data primary index (member_sk, dest_member_sk)
on commit preserve rows
"""

pandas.io.sql.execute(sql8_tl_extend, cxn)

sql8_count = """
select count(distinct dest_member_sk) as tle_cxns 
    , count(distinct case when a.seniority_2_sk >= 6 then dest_member_sk end) as tle_dm_cxns
from teamlink_extend a

"""

get_tle_cxns = pandas.read_sql(sql8_count, cxn)

get_tle_cxns.head()

In [ ]:
sql10_avg_dm_cxns = """
select count(a.member_sk) / {0} as avg_dm_cxns --edit number of sales reps
from com_sp_cxns a

where a.seniority_2_sk >= 6

""".format(num_sp)

get_avg_dm_cxns = pandas.read_sql(sql10_avg_dm_cxns, cxn)

get_avg_dm_cxns.head()


In [ ]:
sql11_pre_post_cxns = """
select 'Pre ' as time_period
    , count(distinct dest_member_sk) as cxns
    , dm_cxns
from sn_cxns c
    inner join all_sn s
    on c.member_sk = s.member_sk
    
    left join (
        select count(distinct dest_member_sk) as dm_cxns
        from sn_cxns c
        
        inner join all_sn s
        on c.member_sk = s.member_sk

        inner join dm_stt.v_om_segment o
        on c.dest_member_sk = o.member_sk
            and o.seniority_2_sk >= 6
        where c.date_sk between s.onboard_complete_dt - days_onboarded and s.onboard_complete_dt -1
    ) a on 1=1
    
where c.date_sk between s.onboard_complete_dt - days_onboarded and s.onboard_complete_dt - 1

group by 1, 3

union all 

select 'Post'
    , count(distinct dest_member_sk) 
    , dm_cxns
from sn_cxns c
    inner join all_sn s
    on c.member_sk = s.member_sk
    
    left join (
        select count(distinct dest_member_sk) as dm_cxns
        from sn_cxns c
        inner join all_sn s
        on c.member_sk = s.member_sk
        
        inner join dm_stt.v_om_segment o
        on c.dest_member_sk = o.member_sk
            and o.seniority_2_sk >= 6
        where c.date_sk >= s.onboard_complete_dt
    ) a on 1=1
where c.date_sk >= s.onboard_complete_dt
group by 1, 3
order by 1 desc
;
"""

get_pre_post_cxns = pandas.read_sql(sql11_pre_post_cxns, cxn)

get_pre_post_cxns.head()

#avg_dm_cxns = get_avg_dm_cxns["avg_dm_cxns"].iloc[0]

#print avg_dm_cxns

In [ ]:
sql12_outbound_pvs = """
create volatile table outbound_pvs as (
    select a.member_sk
        , p.viewee_member_sk
        , p.profile_views
        , p.network_distance
        , p.date_sk
    from dwh.v_agg_dly_mmb_profile_views_vy p
        inner join all_sn a
            on a.member_sk = p.viewer_member_sk 
    where p.is_nonself_view = 1
)   
with data primary index (member_sk, viewee_member_sk) 
on commit preserve rows
;
"""

pandas.io.sql.execute(sql12_outbound_pvs, cxn)

In [ ]:
sql13_pre_post_pvs = """
select 'Pre ' as time_period
    , sum(profile_views) as profile_views
    , sum(case when o.seniority_2_sk >= 6 then profile_views end) as dm_pvs
from outbound_pvs p
    inner join all_sn s
    on p.member_sk = s.member_sk
    
    inner join dm_stt.v_om_segment o
    on p.viewee_member_sk = o.member_sk
    
where p.date_sk between s.onboard_complete_dt - days_onboarded and s.onboard_complete_dt - 1
group by 1

union all 

select 
    'Post'
    , sum(profile_views) 
    , sum(case when o.seniority_2_sk >= 6 then profile_views end)
from outbound_pvs p
    inner join all_sn s
    on p.member_sk = s.member_sk
    
    inner join dm_stt.v_om_segment o
    on p.viewee_member_sk = o.member_sk
    
where p.date_sk >= s.onboard_complete_dt
            
group by 1
order by 1 desc

"""
get_pre_post_pvs = pandas.read_sql(sql13_pre_post_pvs, cxn)

get_pre_post_pvs.head()

In [ ]:
sql14_pre_post_srch = """
select  'Pre ' as time_period
    , sum(d.search_cnt) as searches
from dwh.v_agg_daily_search d 
    inner join all_sn s
    on d.member_sk = s.member_sk
where d.date_sk between s.onboard_complete_dt - s.days_onboarded and s.onboard_complete_dt - 1
group by 1

union all

select 'Post '
    ,sn_searches
from (
    select sum(SN_SEARCH + LI_SEARCH_BY_SN ) 
        as sn_searches
    from dwh.v_agg_daily_lss_seat_stats l 
        inner join all_sn s
            on l.lss_seat_id = s.lss_seat_id
    where l.date_sk between s.onboard_complete_dt and current_date
) b
group by 1
;
"""

get_pre_post_srch = pandas.read_sql(sql14_pre_post_srch, cxn)

get_pre_post_srch.head()

sql15_pre_temp = """
select count(a.member_sk) as count_sn
    , avg(a.SCORE_SOCIAL_BRAND) as social_brand
    , avg(a.score_right_people) as right_people
    , avg(a.score_insights) as insights
    , avg(a.score_relationships) as relationships
    , avg(a.ssi) as ssi
from dwh.v_agg_monthly_all_member_ssi a

inner join all_sn b
on a.member_sk = b.member_sk

inner join dwh.v_dim_date c
on c.month_begin_sk = a.month_begin_sk
    and c.date_sk = (select min(b.onboard_complete_dt) from all_sn)
"""

print (sql15_pre_temp)
get_pre_ssi_temp = pandas.read_sql(sql15_pre_temp, cxn)

get_pre_ssi_temp.head()

sql15_pre_post_ssi = """
select 'Pre ' as time_period
    , count(b.member_sk) as count_sn
    , avg(a.pre_score_social_brand) as social_brand
    , avg(a.pre_score_right_people) as right_people
    , avg(a.pre_score_insights) as insights
    , avg(a.pre_score_relationships) as relationships
    , avg(a.pre_ssi) as ssi
    
from DWH.V_AGG_MBR_COMP_PRE_SSI a

inner join all_sn b 
on b.member_sk = a.member_sk

inner join dm_stt.v_om_segment c 
on c.member_sk = b.member_sk

inner join dwh.v_dim_company d
on c.std_company_sk = d.company_sk 
    and d.company_id in ({0})

union all

select 'Post' as time_period
    , count(a.member_sk) 
    , avg(b.score_social_brand_v3) 
    , avg(b.score_right_people_v3) 
    , avg(b.score_insights_v3) 
    , avg(b.score_relationships_v3) 
    , avg(b.ssi_v3)

from DWH.V_AGG_CURR_SALES_SSI b

inner join all_sn a 
on b.member_sk = a.member_sk
    
;

""".format(company_id)

print (sql15_pre_post_ssi)
get_pre_post_ssi = pandas.read_sql(sql15_pre_post_ssi, cxn)

get_pre_post_ssi.head()

sql_15_raw_ssi = """
select 'Pre ' as time_period
    , b.member_sk
    , a.pre_score_social_brand
    , a.pre_score_right_people
    , a.pre_score_insights
    , a.pre_score_relationships
    , a.pre_ssi
    
from DWH.V_AGG_MBR_COMP_PRE_SSI a

inner join all_sn b 
on b.member_sk = a.member_sk

inner join dm_stt.v_om_segment c 
on c.member_sk = b.member_sk

inner join dwh.v_dim_company d
on c.std_company_sk = d.company_sk 
    and d.company_id in (3681456)

union all

select 'Post' as time_period
    , a.member_sk
    , b.score_social_brand_v3
    , b.score_right_people_v3
    , b.score_insights_v3
    , b.score_relationships_v3
    , b.ssi_v3

from DWH.V_AGG_CURR_SALES_SSI b

inner join all_sn a 
on b.member_sk = a.member_sk
    
;
"""

In [ ]:
sql16_saved_leads = """
select count(a.lead_member_id) as leads_saved

from dwh.v_dim_lss_prospects a
    inner join all_sn s 
       on s.lss_seat_id = a.lss_seat_id
    inner join dm_stt.v_om_segment o
       on a.lead_member_id = o.member_sk
where a.active = 'Y'
   and a.current_flag = 'Y'
    and a.state = 'A'
"""

get_saved_leads = pandas.read_sql(sql16_saved_leads, cxn)

get_saved_leads.head()

In [ ]:
sql17_saved_accounts = """
select count(c.company_sk) as accounts_saved
from dwh.v_dim_lss_biz_prospects b

inner join all_sn s
on b.lss_seat_id = s.lss_seat_id

inner join dwh.v_dim_company c
on b.acct_company_id = c.company_id

where state in ( 'A', 'I')
    and b.active = 'Y'
    and current_flag = 'Y'
"""

get_saved_accounts = pandas.read_sql(sql17_saved_accounts, cxn)

get_saved_accounts.head()

In [ ]:
sql18_insights = """
select 
    sum(a.lead_recommendations) as lead_recommendations
    , sum(a.account_share_article_cnt) as account_shares
    , sum(a.company_mention_news_cnt) as mentions_in_news
    , sum(a.lead_share_article_cnt) as lead_shares
    , sum(a.new_position_fill_cnt) as new_jobs
    , sum(a.account_share_content_cnt) as account_content_share
    , sum(insight_emails_sent) as insights_emails_sent
    , sum(a.sn_search) as sn_search_only_ratio
    , sum(a.li_search_by_sn) as li_search_only_ratio
    , sum(a.total_profile_views) as outbound_profile_views

from dwh.v_agg_daily_lss_seat_stats a
inner join all_sn b
on a.lss_seat_id = b.lss_seat_id

where date_sk >= b.onboard_complete_dt
;
"""

get_insights = pandas.read_sql(sql18_insights, cxn)

get_insights.head()

In [ ]:
#Outputs
#Connections
avg_dm_cxns = get_avg_dm_cxns["avg_dm_cxns"].iloc[0]
sn_cxns = get_sn_cxns["sn_cxns"].iloc[0]
sn_dm_cxns = get_sn_cxns["sn_dm_cxns"].iloc[0]

com_sp_cxns = get_com_sp_cxns["com_sp_cxns"].iloc[0]
com_sp_dm_cxns = get_com_sp_cxns["com_sp_dm_cxns"].iloc[0]
all_emp_cxns = get_all_emp_cxns["all_emp_cxns"].iloc[0]
all_emp_dm_cxns = get_all_emp_cxns["all_emp_dm_cxns"].iloc[0]
tle_cxns = get_tle_cxns["tle_cxns"].iloc[0]
tle_dm_cxns = get_tle_cxns["tle_dm_cxns"].iloc[0]
pre_cxns = get_pre_post_cxns["cxns"].iloc[0]
pre_dm_cxns = get_pre_post_cxns["dm_cxns"].iloc[0]
post_cxns = get_pre_post_cxns["cxns"].iloc[1]
post_dm_cxns = get_pre_post_cxns["dm_cxns"].iloc[1]
dm_cxn_ratio = round(float(post_dm_cxns)/pre_dm_cxns, 1)
cxn_ratio = round(float(post_cxns)/pre_cxns, 1)
#Profile Views
pre_pvs = get_pre_post_pvs["profile_views"].iloc[0]
post_pvs = get_pre_post_pvs["profile_views"].iloc[1]
pct_inc_pvs = round((float(post_pvs - pre_pvs) / float(pre_pvs))*100, 1)

#Searches
pre_srch = get_pre_post_srch["searches"].iloc[0]
post_srch = get_pre_post_srch["searches"].iloc[1]
pct_inc_srch = round((float(post_srch - pre_srch) / float(pre_srch))*100, 1)

#Saved Leads & Accounts
leads_saved = get_saved_leads["leads_saved"].iloc[0]
accounts_saved = get_saved_accounts["accounts_saved"].iloc[0]

#SSI
pre_brand = get_pre_ssi_temp["social_brand"].iloc[0]
pre_people = get_pre_ssi_temp["right_people"].iloc[0]
pre_insights = get_pre_ssi_temp["insights"].iloc[0]
pre_relationships = get_pre_ssi_temp["relationships"].iloc[0]
pre_ssi = get_pre_ssi_temp["ssi"].iloc[0]

#pre_brand = get_pre_post_ssi["social_brand"].iloc[0]
#pre_people = get_pre_post_ssi["right_people"].iloc[0]
#pre_insights = get_pre_post_ssi["insights"].iloc[0]
#pre_relationships = get_pre_post_ssi["relationships"].iloc[0]
#pre_ssi = get_pre_post_ssi["ssi"].iloc[0]
post_brand = get_pre_post_ssi["social_brand"].iloc[1]
post_people = get_pre_post_ssi["right_people"].iloc[1]
post_insights = get_pre_post_ssi["insights"].iloc[1]
post_relationships = get_pre_post_ssi["relationships"].iloc[1]
post_ssi = get_pre_post_ssi["ssi"].iloc[1]

#Insights
shares_surfaced = get_insights["account_shares"].iloc[0] + get_insights["account_content_share"].iloc[0] + get_insights["lead_shares"].iloc[0]
news_mentions = get_insights["mentions_in_news"].iloc[0]
insights_emails = get_insights["insights_emails_sent"].iloc[0]

          

In [ ]:
from pptx.chart.data import ChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.util import Inches


# define chart data ---------------------
searches_chart = ChartData()
searches_chart.categories = get_pre_post_srch["time_period"]
#chart_data.add_series('6 Months Prior', (get_cxns_output["pre_target_cxns"]))
searches_chart.add_series('Pre to Post Searches', (get_pre_post_srch["searches"]))

x, y, cx, cy = Inches(1.98), Inches(1.43), Inches(6.04), Inches(3.68)

# define chart data ---------------------
pvs_chart = ChartData()
pvs_chart.categories = get_pre_post_pvs["time_period"]
#chart_data.add_series('6 Months Prior', (get_cxns_output["pre_target_cxns"]))
pvs_chart.add_series('Pre to Post Profile Views', (get_pre_post_pvs["profile_views"]))

cxns_chart = ChartData()
cxns_chart.categories = get_pre_post_cxns["time_period"]
if dm_cxn_ratio > cxn_ratio :
    cxns_chart.add_series('Pre to Post Profile Views', (get_pre_post_cxns["dm_cxns"]))
    print "Using DM connections as " + str(dm_cxn_ratio) + " is higher than " + str(cxn_ratio)
    str_cxn_ratio = "Your team has accelerated their network growth by connecting to {0}X more decision makers during the pilot".format(
        dm_cxn_ratio)
else :
    cxns_chart.add_series('Pre to Post Profile Views', (get_pre_post_cxns["cxns"]))
    print "Using all connections as " + str(cxn_ratio) + " is higher than " + str(dm_cxn_ratio)
    str_cxn_ratio = "Your team has accelerated their network growth by connecting {0}X more during the pilot".format(
        cxn_ratio)

In [ ]:
import os
from pptx import Presentation
from pptx.enum.chart import XL_CHART_TYPE
from pptx.util import Inches
from pptx.enum.chart import XL_TICK_MARK
from pptx.util import Pt
from pptx.dml import line
from pptx.enum.chart import XL_TICK_LABEL_POSITION
from pptx.dml.color import RGBColor
from pptx.enum.chart import XL_LABEL_POSITION
import datetime


def create_ppt(input, output):#, report_data, chart):
    prs = Presentation(input)
    # Use the output from analyze_ppt to understand which layouts and placeholders
    # Create a title slide first
    
    #Activity Slide
    activity_slide =  prs.slide_layouts[1]
    
    slide1 = prs.slides.add_slide(activity_slide)
    title1 = slide1.shapes.title
    subtitle1 = slide1.placeholders[10]
    subtitle2 = slide1.placeholders[11]
    subtitle3 = slide1.placeholders[12]
    subtitle4 = slide1.placeholders[13]
    subtitle5 = slide1.placeholders[14]
    subtitle6 = slide1.placeholders[15]
    
    #pic = subtitle0.insert_picture(bar_img_path)
    subtitle1.text = "{0}".format(post_srch)
    subtitle2.text = "{0}".format(post_pvs)
    subtitle3.text = "{0}".format(leads_saved)
    subtitle4.text = "{0}".format(accounts_saved)
    subtitle5.text = "{0}".format(post_cxns)
    subtitle6.text = "{0}".format(post_dm_cxns)
    
    #Searches Slide
    searches_slide = prs.slide_layouts[2]
    slide2 = prs.slides.add_slide(searches_slide)
    subtitle7 = slide2.placeholders[11]
    subtitle7.text = "Your team has accelerated their prospecting activity by performing {0}X more searches during the pilot".format(
        round(float(post_srch)/pre_srch, 1))
    
    graphic_frame = slide2.shapes.add_chart(
    XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, searches_chart
    )
    chart = graphic_frame.chart

    category_axis = chart.category_axis
    category_axis.has_major_gridlines = False 
    category_axis.has_minor_gridlines = False
    category_axis.major_tick_mark = XL_TICK_MARK.NONE
    category_axis.minor_tick_mark = XL_TICK_MARK.NONE
    category_axis.tick_labels.font.size = Pt(10)
    category_axis.tick_labels.font.color.rgb=RGBColor(0xff, 0xff, 0xff)
    
    value_axis = chart.value_axis
    value_axis.minimum_scale = 0
    value_axis.maximum_scale = None
    value_axis.has_major_gridlines = False 
    value_axis.has_minor_gridlines = False
    value_axis.visible = False
    #value_axis.tick_labels.font.size = Pt(10)

    plot = chart.plots[0] 
    plot.has_data_labels = True
    data_labels = plot.data_labels

    data_labels.font.size = Pt(10)
    data_labels.font.color.rgb = RGBColor(0xff, 0xff, 0xff)
    data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
    #tick_labels.font.color.rgb=RGBColor(0x0A, 0x42, 0x80)
    
    #Profile Views Slide
    pvs_slide = prs.slide_layouts[3]
    slide3 = prs.slides.add_slide(pvs_slide)
    subtitle8 = slide3.placeholders[11]
    subtitle8.text = "Your team has accelerated their qualifying activity by viewing {0}X more profiles during the pilot".format(
        round(float(post_pvs)/pre_pvs, 1))
    
    graphic_frame = slide3.shapes.add_chart(
    XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, pvs_chart
    )
    chart = graphic_frame.chart

    category_axis = chart.category_axis
    category_axis.has_major_gridlines = False 
    category_axis.has_minor_gridlines = False
    category_axis.major_tick_mark = XL_TICK_MARK.NONE
    category_axis.minor_tick_mark = XL_TICK_MARK.NONE
    category_axis.tick_labels.font.size = Pt(10)
    category_axis.tick_labels.font.color.rgb = RGBColor(0xff, 0xff, 0xff)
    
    value_axis = chart.value_axis
    value_axis.minimum_scale = 0
    value_axis.maximum_scale = None
    value_axis.has_major_gridlines = False 
    value_axis.has_minor_gridlines = False
    value_axis.visible = False
    #value_axis.tick_labels.font.size = Pt(10)

    plot = chart.plots[0] 
    plot.has_data_labels = True
    data_labels = plot.data_labels

    data_labels.font.size = Pt(10)
    data_labels.font.color.rgb = RGBColor(0xff, 0xff, 0xff)
    data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
    #tick_labels.font.color.rgb=RGBColor(0x0A, 0x42, 0x80)
    
    #Connections Slide
    cxns_slide = prs.slide_layouts[4]
    slide4 = prs.slides.add_slide(cxns_slide)
    subtitle9 = slide4.placeholders[11]
    subtitle9.text = "{0}".format(str_cxn_ratio)
    
    graphic_frame = slide4.shapes.add_chart(
    XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, cxns_chart
    )
    chart = graphic_frame.chart

    category_axis = chart.category_axis
    category_axis.has_major_gridlines = False 
    category_axis.has_minor_gridlines = False
    category_axis.major_tick_mark = XL_TICK_MARK.NONE
    category_axis.minor_tick_mark = XL_TICK_MARK.NONE
    category_axis.tick_labels.font.size = Pt(10)
    
    value_axis = chart.value_axis
    value_axis.minimum_scale = 0
    value_axis.maximum_scale = None
    value_axis.has_major_gridlines = False 
    value_axis.has_minor_gridlines = False
    value_axis.visible = False
    #value_axis.tick_labels.font.size = Pt(10)

    plot = chart.plots[0] 
    plot.has_data_labels = True
    data_labels = plot.data_labels

    data_labels.font.size = Pt(10)
    data_labels.font.color.rgb =RGBColor(0xff, 0xff, 0xff)
    data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
    
    #Insights Delivered Slide
    insights_slide = prs.slide_layouts[5]
    slide5 = prs.slides.add_slide(insights_slide)
    subtitle30 = slide5.placeholders[10]
    subtitle31 = slide5.placeholders[11]
    subtitle32 = slide5.placeholders[12]
    subtitle33 = slide5.placeholders[13]

    subtitle31.text = "Saving leads and accounts allowed Sales Navigator to proactively keep your reps informed on the people and companies that matter to them"
    subtitle30.text = "{0}".format(shares_surfaced)
    subtitle32.text = "{0}".format(news_mentions)
    subtitle33.text = "{0}".format(insights_emails)
    

    network_slide = prs.slide_layouts[6]
    slide7 = prs.slides.add_slide(network_slide)
    subtitle20 = slide7.placeholders[11]
    subtitle21 = slide7.placeholders[12]
    
    subtitle20.text = "{0}".format(tle_dm_cxns)
    subtitle21.text = "{0}".format(avg_dm_cxns)

    prs.save(output)

#Old SSI Stuff

#SSI Changes as a whole point %
inc_social_brand = int(round(((post_brand - pre_brand) / pre_brand)*100, 1))
inc_right_people = int(round(((post_people - pre_people) / pre_people)*100, 1))
inc_insights = int(round(((post_insights - pre_insights) / pre_insights)*100, 1))
inc_relationships = int(round(((post_relationships - pre_relationships) / pre_relationships)*100, 1))
inc_ssi = int(round(((post_ssi - pre_ssi) / pre_ssi)*100, 1))

if inc_social_brand > 0 :
    inc_social_brand_text = "{0}% increase".format(inc_social_brand)
elif inc_social_brand < 0:
    inc_social_brand_text = "{0}% decrease".format(inc_social_brand)
else :
    inc_social_brand_text = "No change"
    
if inc_right_people > 0 :
    inc_right_people_text = "{0}% increase".format(inc_right_people)
elif inc_right_people < 0:
    inc_right_people_text = "{0}% decrease".format(inc_right_people)
else :
    inc_right_people_text = "No change"
    
if inc_insights > 0 :
    inc_insights_text = "{0}% increase".format(inc_insights)
elif inc_insights < 0:
    inc_insights_text = "{0}% decrease".format(inc_insights)
else :
    inc_insights_text = "No change"

if inc_relationships > 0 :
    inc_relationships_text = "{0}% increase".format(inc_relationships)
elif inc_relationships < 0:
    inc_relationships_text = "{0}% decrease".format(inc_relationships)
else :
    inc_relationships_text = "No change"

if inc_ssi > 0 :
    inc_ssi_text = "{0}% increase".format(inc_ssi)
elif inc_ssi < 0:
    inc_ssi_text = "{0}% decrease".format(inc_ssi)
else :
    inc_ssi_text = "No change"
    

print "inc_social_brand: " + str(inc_social_brand_text)
print "inc_right_people: " + str(inc_right_people_text)
print "inc_insights: " + str(inc_insights_text)
print "inc_relationships: " + str(inc_relationships_text)
print "inc_ssi: " + str(inc_ssi_text)
print '\n'
#SSI Slide
    ssi_slide = prs.slide_layouts[6]
    slide6 = prs.slides.add_slide(ssi_slide)
    subtitle23 = slide6.placeholders[20]
    subtitle10 = slide6.placeholders[10]
    subtitle11 = slide6.placeholders[11]
    subtitle12 = slide6.placeholders[12]
    subtitle13 = slide6.placeholders[13]
    subtitle14 = slide6.placeholders[14]
    subtitle15 = slide6.placeholders[15]
    subtitle16 = slide6.placeholders[16]
    subtitle17 = slide6.placeholders[17]
    subtitle18 = slide6.placeholders[18]
    subtitle19 = slide6.placeholders[19]
    subtitle24 = slide6.placeholders[21]
    subtitle25 = slide6.placeholders[22]
    subtitle26 = slide6.placeholders[23]
    subtitle27 = slide6.placeholders[24]
    subtitle28 = slide6.placeholders[25]
    subtitle29 = slide6.placeholders[26]
    
    subtitle23.text = "Social Selling Adoption Metrics"
    subtitle10.text = "{0}".format(round(pre_brand, 1))
    subtitle11.text = "{0}".format(round(pre_people, 1))
    subtitle12.text = "{0}".format(round(pre_insights, 1))
    subtitle13.text = "{0}".format(round(pre_relationships, 1))
    subtitle14.text = "{0}".format(round(pre_ssi, 1))
    subtitle15.text = "{0}".format(round(post_brand, 1))
    subtitle16.text = "{0}".format(round(post_people, 1))
    subtitle17.text = "{0}".format(round(post_insights, 1))
    subtitle18.text = "{0}".format(round(post_relationships, 1))
    subtitle19.text = "{0}".format(round(post_ssi, 1))
    subtitle24.text = "*Before pilot was a snapshot of SSI as of users’ onboard date. After pilot was a snapshot of SSI as at {0}".format(
        datetime.date.today())
    subtitle25.text = "{0}".format(inc_social_brand_text)
    subtitle26.text = "{0}".format(inc_right_people_text)
    subtitle27.text = "{0}".format(inc_insights_text)
    subtitle28.text = "{0}".format(inc_relationships_text)
    subtitle29.text = "{0}".format(inc_ssi_text)

In [ ]:
create_ppt(template_location, result_location + result_filename + '.pptx')

In [ ]:
from pptx import Presentation

def analyze_ppt(input, output):
    """ Take the input file and analyze the structure.
    The output file contains marked up information to make it easier
    for generating future powerpoint templates.
    """
    prs = Presentation(input)
    # Each powerpoint file has multiple layouts
    # Loop through them all and  see where the various elements are
    for index, _ in enumerate(prs.slide_layouts):
        slide = prs.slides.add_slide(prs.slide_layouts[index])
        # Not every slide has to have a title
        try:
            title = slide.shapes.title
            title.text = 'Title for Layout {}'.format(index)
        except AttributeError:
            print("No Title for Layout {}".format(index))
        # Go through all the placeholders and identify them by index and type
        for shape in slide.placeholders:
            if shape.is_placeholder:
                phf = shape.placeholder_format
                # Do not overwrite the title which is just a special placeholder
                try:
                    if 'Title' not in shape.text:
                        shape.text = 'Placeholder index:{} type:{}'.format(phf.idx, shape.name)
                except AttributeError:
                    print("{} has no text attribute".format(phf.type))
                print('{} {}'.format(phf.idx, shape.name))
    prs.save(output)

In [ ]:
#analyze_ppt('C:\\Users\\kachong\\Google Drive\\Notebooks\\Pilot_Review.pptx', result_location + result_filename + '.pptx')